Alumno: Ignacio Carol Lugones


Padron: 100073

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 100): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step), rotation=90)
    plt.show() 

# Calibracion de parametros

Definimos el tamaño del tablero, sabiendo que cada celda del ajedrez medira 28 mm

In [ ]:
chessBoardSize  = (8, 6)
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)
objp = objp * 28

Cargamos imagenes y dibujamos

In [ ]:
img_fnames = glob('./imagenes/img_cal_set1/*')
imgsGray = []
imgsColor = []
imgNames = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    imgNames.append(imgName)


In [ ]:
def findCorners(imgsColor, imgsGray, plot=True, maxCount = 30, epsilon = 0.001, flag=cv.CALIB_CB_ADAPTIVE_THRESH):
    imgPoints = []
    objPoints = []
    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_MAX_ITER, maxCount, epsilon)
    cb_flags = flag
    i = 1
    for imgColor, imgGray in zip(imgsColor, imgsGray):
        imgColor = imgColor.copy()
        ret, corners = cv.findChessboardCorners(imgGray, chessBoardSize, flags=cb_flags)
        if ret:
            objPoints.append(objp)
            corners_subp = cv.cornerSubPix(imgGray, corners, (5, 5), (-1, -1), criteria)
            imgPoints.append(corners_subp)
            cv.drawChessboardCorners(imgColor, chessBoardSize, corners_subp, ret)
            if plot:
                print(i)
                plotter(imgColor)
        i+=1
    return imgPoints, objPoints

def calibrateCamera(objPoints, imgPoints, widthAndHeight, returnMatrix=False):
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objPoints, imgPoints, widthAndHeight, None, None, flags = cv.CALIB_ZERO_TANGENT_DIST)
    print('Camera Matrix: \n{}'.format(mtx))
    print('\nDistortion Coefficients: \n{}\n'.format(dist))
    if returnMatrix:
        return mtx, dist

def getIntrinsecParams(imgsColor, imgsGray, printImg = False):
    imgPoints, objPoints = findCorners(imgsColor, imgsGray, printImg)
    height, width = imgsGray[0].shape
    return calibrateCamera(objPoints, imgPoints, (width,height), returnMatrix=True)

In [ ]:
mtx, dist = getIntrinsecParams(imgsColor, imgsGray, True)


In [ ]:
#Set 2
chessBoardSize  = (8, 6)
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)
objp = objp * 28
img_fnames = glob('./imagenes/img_cal_set2/*')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
mtx, dist = getIntrinsecParams(imgsColor, imgsGray, True)

Se ve que se tiene que usar el set 1 en vez del set dos porque el fx y el fy del set 1 son mas similares entre si, lo que conlleva que la matriz de parametros intrinsecos del set 1 esta mejor generada que la del set 2

In [ ]:
# reload set1
chessBoardSize  = (8, 6)
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)
objp = objp * 28
img_fnames = glob('./imagenes/img_cal_set1/*')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    imgNames.append(imgName)
mtx, dist = getIntrinsecParams(imgsColor, imgsGray)

# 2, sacamos parametros extrinsecos e implicitamente la terna de trabajo

In [ ]:
#Levantamos la imagen para calibracion de parametros extrinsecos
objp = np.zeros((np.prod(chessBoardSize), 3),  dtype=np.float32)
objp[:, :2] = np.mgrid[0:chessBoardSize[0], 0:chessBoardSize[1]].T.reshape(-1, 2)
objp = objp * 28
img_fnames = glob('./imagenes/img_bloques/imgCalExtr.png')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
imgPoints, objPoints = findCorners(imgsColor, imgsGray, True)

In [ ]:
ret, rvecs, t = cv.solvePnP(objPoints[0], imgPoints[0], mtx, dist, useExtrinsicGuess=False)
rotation = cv.Rodrigues(rvecs)[0]
print(t)
print(rotation)

Al tener una sola camara, en escenarios futuros consideraremos que la terna z debera ser 0, ya que no se puede estimar la profundidad correctamente.

# 3 Busqueda de bloques

In [ ]:
def checkCenter(c):
    return (c[0] > 92 and c[0] < 500) and (c[1] > 20 and c[1] < 468)
def processImage(img):
    centers = []
    boxes = []
    img = cv.medianBlur(img, 5)
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    ret, img_bin = cv.threshold(imgGray, 30, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    
    kernel = np.ones((2,2), np.uint8)
    opening = cv.morphologyEx(img_bin, cv.MORPH_OPEN, kernel) # removes noice
    
    kernel = np.ones((2,2), np.uint8)
    ending = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel) #fill blanks
    
    contours, hierarchy = cv.findContours(ending, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_L1)
    
    for cnt in contours:
        rect = cv.minAreaRect(cnt)
        center = rect[0]
        dim = rect[1]
        rot = rect[2]
        area = cv.contourArea(cnt)
        if checkCenter(center) and area > 3000 and area < 100000:
            box = cv.boxPoints(rect)
            box = np.int0(box)
            if box[0][0] < 50 or box[0][1] < 20:
                continue
            cv.circle(img, (int(center[0]), int(center[1])), 1, (0,0,255), 2)
            cv.drawContours(img, [box], 0, (255,0,0), 2)
            plt.imshow(img)
            plt.show()
            print(f"rotation: {rot}, dimensions are: {dim}, center at: {center}")
            centers.append(center)
            boxes.append(box)
    return centers, boxes


In [ ]:
imgNames = glob('./imagenes/img_bloques/imgBloque*')
imgs = {}
for name in imgNames:
    img = cv.imread(name)
    print(f'\nnow checking img {name}')
    imgs[name] = processImage(img)
# centers 0, boxes 1

In [ ]:
imgs

In [ ]:
import re
def printResults(results):
    print('RESULTS')
    for key, value in results.items():
        imgNumber = re.findall(r'\d+', key)
        if len(imgNumber) == 0:
            print(key)
            continue #border case
        print(f'Resultados para imagen {imgNumber[-1]}')
        amountOfBlocks = len(value[0])
        print(f'bloques encontrados: {amountOfBlocks}')
        for i in range(amountOfBlocks):
            print(f'[{i + 1}]--->\ncenter: {value[0][i]};\nbox: {value[1][i]}')
        print('-'*100)
            
    

In [ ]:
printResults(imgs)

# 4 Propuesta de validacion de algoritmo:
## Primer posibilidad
Una posible validacion del mismo deberia ser una empirica, esto significa el ver con nuestras propias mediciones a la hora de sacar la foto que lo que nos indique el programa sea correcto.
Para esto primero tendremos que sacar el punto de donde se considere nuestro Xw = 0 y nuestro Yw = 0. Esto es relativamente facil de hacer ya que tenemos todos los parametros para sacarlo. Esto tambien se debe comprobar con lo esperado a traves de las calibraciones hechas.

Luego de hacer esto, tendremos que ubicar un bloque en el tablero y sacar la foto para que la camara indique en que Xw e Yw esta para ella, una vez obtenido esto debemos nosotros medir con algun instrumento de medicion preciso donde esta ubicado este elemento para nosotros con respecto a nuestra terna Xw e Yw, si estos valores coinciden entonces el algoritmo ha sido validado, en caso contrario es que el algoritmo fallo.
## Segunda posibilidad
Como segunda posibilidad de validacion, lo que podemos hacer es nosotros forzar una terna xyz a traves de una imagen de calibracion nuestra conocida, una vez hecha esta calibracion, podemos posicionar un bloque, o una imagen de un bloque, sobre nuestro tablero en una posicion conocida, la cual tengamos exactitud de que posicion esta con respecto a esta terna.
Con esto, lo compararemos con los datos obtenidos por nuestro algoritmo y asi podremos validar la precision del mismo.

# 5 Pasaje a mm, aca tambien termina el 3 con sus coordenadas en mm al centro

### Como se calcula las coordenadas
s * [uv1] = K * [R|T] *XwYwZw = k * (R XwYwZw + t)

==> R^-1 * (K^-1 * s * [uv1] - t) = XwYwZw


K: intrinsic params

R: rotation

t: translation

s: correcter factor

given that Zw should be 0 because we don't have more cameras, Zw should be 0

for that to happen the following should be true:

s * R^-1 * K^-1  * [uv1] - R^-1 * t = XwYw0


More specifically, what should happen is that the result of the last row should be 0

in that case to accomplish that condition:

s = sum of last row of R^-1 * t / sum of last row of R ^-1 * K^-1 * [uv1]

In [ ]:
def getCoord(center):
    # s * [uv1] = K * [R|T] *XwYwZw = k * (R XwYwZw + t)
    # ==> R^-1 * (K^-1 * [uv1] - t) = XwYwZw
    # K intrinsic params
    # R rotation
    # t translation
    # given that Zw should be 0 because we don't have more cameras
    uv = np.array([[center[0],center[1],1]], dtype=np.float).T
    iK = np.linalg.inv(mtx)
    iR = np.linalg.inv(rotation)
    numerator = iR.dot(t)
    denominator = iR.dot(iK).dot(uv)
    s = sum(numerator[2]) / sum(denominator[2])
    first = (s * iK.dot(uv) - t)
    return iR.dot(first)

def getZero():
    # given last ecuation, if i want to get the 0 0 0 we should use the first equation, or just multiply k by t:
    s = 715.5301268026278 # extracted from an average
    return mtx.dot(t) / s


def f_dist(x):
    k1 = dist[0][0]
    xn = x[0:2]
    r_sqr = xn[0] ** 2 + xn[1] ** 2
    xd = xn * (1+k1 * r_sqr)
    return np.vstack((xd,1))

def getCoordIter(c):
    Kinv = np.linalg.inv(mtx)
    Rt = np.hstack((rotation, t))
    Rt_z0 = np.delete(Rt, 2, 1)
    RtInv = np.linalg.inv(Rt_z0)
    iterTimes = 100
    uv = np.matrix([[c[0]], [c[1]], [1]])
    x_dist = Kinv.dot(uv)
    dx = x_dist - f_dist(x_dist)
    X = x_dist - dx
    for _ in range(iterTimes):
        dx = x_dist - f_dist(X)
        X = x_dist - dx
    return RtInv.dot(X)

def getCoordHomo(center):
    uv = np.array([[center[0],center[1],1]], dtype=np.float).T
    r = rotation.T[:2].T
    extrinsic = np.append(r, t, axis=1)
    iK = np.linalg.inv(mtx.dot(extrinsic))
    return iK.dot(uv)

In [ ]:
def getDimension(v):
    '''The idea of this function, is receiving a box, return a the real dimension of the rectangle'''
    v = [getCoord(x)[:2] for x in v]
    w = np.sqrt((v[1][0]-v[0][0])**2+(v[1][1]-v[0][1])**2)
    h = np.sqrt((v[3][0]-v[0][0])**2+(v[3][1]-v[0][1])**2)
    if w > 140 or h > 140: # for the border case of having the diagonal
        if w > h:
            w = np.sqrt((v[1][0]-v[3][0])**2+(v[1][1]-v[3][1])**2)
        else:
            h = np.sqrt((v[1][0]-v[3][0])**2+(v[1][1]-v[3][1])**2)
    if w > h:
        return (w, h)
    else:
        return (h, w)

In [ ]:
realValues = {}
for k, v in imgs.items():
    realValues[k] = []
    arr = False
    for center in v[0]:
        l = getCoord(center)
        if not arr:
            arr = [np.array([l[0][0], l[1][0]])]
        else:
            arr += [np.array([l[0][0], l[1][0]])]
    realValues[k].append(arr)
    arr = False
    for box in v[1]:
        b = getDimension(box)
        if not arr:
            arr = [np.array([b[0][0], b[1][0]])]
        else:
            arr += [np.array([b[0][0], b[1][0]])]
    realValues[k].append(arr)

In [ ]:
printResults(realValues)

## Calculo de errores

In [ ]:
def plotError(imgsInfo, yLimit=30, step = 10):
    imgNames = sorted(list(imgsInfo.keys()), key=int)
    widthError = [imgsInfo[imgName][1] for imgName in imgNames]
    heightError = [imgsInfo[imgName][0] for imgName in imgNames]
    
    fig, ax = plt.subplots(1,1)
    X_axis = np.arange(len(imgNames))
    ax.bar(X_axis - 0.2, widthError, 0.4, label = 'Width')
    ax.bar(X_axis + 0.2, heightError, 0.4, label = 'Height')
    ax.set_xticklabels(imgNames, rotation=90 if len(imgNames) > 3 else 0)
    ax.set_xticks(X_axis)
    ax.set_yticklabels([i for i in range(0,yLimit,step)])
    ax.set_yticks([i for i in range(0,yLimit,step)])
    ax.set_xlabel('Image', fontsize = 16)
    ax.set_ylabel('Error percentage (%)',fontsize = 16)
    ax.legend()
    ax.grid()
    fig.set_size_inches(16, 8)

In [ ]:
REAL_H = 130
REAL_W = 65
errors = {}
for key, value in realValues.items():
    imgNumber = re.findall(r'\d+', key)[-1]
    h = []
    w = []
    print(f'for image {imgNumber}')
    for i in range(len(value[0])):
        errorW = (abs(REAL_W - value[1][i][1]) / REAL_W) * 100
        errorH = (abs(REAL_H - value[1][i][0]) / REAL_H) * 100
        print(f'[{i + 1}]--->\nerror in h: {errorH}%;\nerror in w: {errorW}%')
        h.append(errorH)
        w.append(errorW)
    errors[imgNumber] = (np.mean(h), np.mean(w))
    print('-'*100)

In [ ]:
plotError(errors, 10, 1)

## Demostracion de donde esta el eje

A continuacion demonstraremos como ubicar el eje Xw e Yw

In [ ]:
axisStarting = getZero()[:2]
axisStarting = (int(axisStarting[0][0]), int(axisStarting[1][0]))
xBox = (axisStarting, (axisStarting[0] + 100, axisStarting[1]))
yBox = (axisStarting, (axisStarting[0], axisStarting[1] + 100))
img = cv.imread('./imagenes/img_bloques/imgBloque15.png') #selected randomly
print(xBox)
cv.line(img, xBox[0], xBox[1], (0, 255, 0), 5)
cv.line(img, yBox[0], yBox[1], (0, 255, 0), 5)
plotter(img)

# Challenge
El primer punto del challenge fue sacado en el punto anterior, con la imagen 20 respectivamente.

In [ ]:
imgNames = glob('./imagenes/img_bloques_desafio/imgBloque*')
imgs = {}
for name in imgNames:
    img = cv.imread(name)
    print(f'\nnow checking img {name}')
    imgs[name] = processImage(img)

In [ ]:
img_fnames = glob('./imagenes/img_bloques_desafio/imgCalExtr.jpg')
imgsGray = []
imgsColor = []
for imgName in img_fnames:
    img = cv.imread(imgName)
    imgsColor.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
imgPoints, objPoints = findCorners(imgsColor, imgsGray, True)
ret, rvecs, t = cv.solvePnP(objPoints[0], imgPoints[0], mtx, dist, useExtrinsicGuess=False)
rotation = cv.Rodrigues(rvecs)[0]
print(t)
print(rotation)

In [ ]:
realValues = {}
for k, v in imgs.items():
    realValues[k] = []
    arr = False
    for center in v[0]:
        l = getCoord(center)
        if not arr:
            arr = [np.array([l[0][0], l[1][0]])]
        else:
            arr += [np.array([l[0][0], l[1][0]])]
    realValues[k].append(arr)
    arr = False
    for box in v[1]:
        b = getDimension(box)
        if not arr:
            arr = [np.array([b[0][0], b[1][0]])]
        else:
            arr += [np.array([b[0][0], b[1][0]])]
    realValues[k].append(arr)

In [ ]:
printResults(realValues)

In [ ]:
REAL_H = 130
REAL_W = 65
errors = {}
for key, value in realValues.items():
    imgNumber = re.findall(r'\d+', key)[-1]
    h = []
    w = []
    print(f'for image {imgNumber}')
    for i in range(len(value[0])):
        errorW = (abs(REAL_W - value[1][i][1]) / REAL_W) * 100
        errorH = (abs(REAL_H - value[1][i][0]) / REAL_H) * 100
        print(f'[{i + 1}]--->\nerror in h: {errorH}%;\nerror in w: {errorW}%')
        h.append(errorH)
        w.append(errorW)
    errors[imgNumber] = (np.mean(h), np.mean(w))
    print('-' * 100)
plotError(errors, 10, 1)